In [67]:
#import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import requests
import hvplot.pandas

from pprint import pprint

# Import API key
from config import geoapify_key

In [68]:
#Import CSV
df = pd.read_csv("output/offences_dates.csv", index_col=0)
df.dropna(how='any', inplace=True)
df.head()


,Reported Date,Suburb - Incident,Postcode - Incident,Offence Level 1 Description,Offence Level 2 Description,Offence Level 3 Description,Offence count,Month,Year
0,2013-07-01,ADELAIDE,5000,OFFENCES AGAINST THE PERSON,ACTS INTENDED TO CAUSE INJURY,Common Assault,2,7,2013
1,2013-07-01,ALDINGA BEACH,5173,OFFENCES AGAINST THE PERSON,ACTS INTENDED TO CAUSE INJURY,Common Assault,1,7,2013
2,2013-07-01,ALDINGA BEACH,5173,OFFENCES AGAINST THE PERSON,ACTS INTENDED TO CAUSE INJURY,Serious Assault not resulting in injury,1,7,2013
3,2013-07-01,BEDFORD PARK,5042,OFFENCES AGAINST THE PERSON,ROBBERY AND RELATED OFFENCES,Blackmail and extortion,1,7,2013
4,2013-07-01,BURTON,5110,OFFENCES AGAINST THE PERSON,OTHER OFFENCES AGAINST THE PERSON,Threatening behaviour,1,7,2013


In [69]:
# Select the 'Suburb - Incident' and 'Postcode - Incident' columns
suburb_postcode_df = df[['Suburb - Incident', 'Postcode - Incident']]

# Drop duplicate rows
unique_suburb_postcode_df = suburb_postcode_df.drop_duplicates()
#Drop duplicate postcodes, just keeping one suburb for each postcode
unique_suburb_postcode_df = unique_suburb_postcode_df.groupby('Postcode - Incident').first().reset_index()
#Add columns for Latitude and Longitude
unique_suburb_postcode_df['Latitude'] = ""
unique_suburb_postcode_df['Longitude'] = ""

unique_suburb_postcode_df.head()

,Postcode - Incident,Suburb - Incident,Latitude,Longitude
0,0872,PUKATJA,,
1,2010,DARLINGHURST,,
2,2148,MARAYONG,,
3,2330,SINGLETON,,
4,2469,BONALBO,,


In [71]:
#get Latitude and Longitude for each postcode via loop

# Create empty lists to store latitude and longitude values
latitude_list = []
longitude_list = []

# Iterate over each row in the DataFrame
for index, row in unique_suburb_postcode_df.iterrows():
    # Fetch suburb and postcode from the row
    suburb = row['Suburb - Incident']
    postcode = row['Postcode - Incident']
    
    # Make a request to Geoapify Geocoding API
    url = f"https://api.geoapify.com/v1/geocode/search?postcode={postcode}&country=Australia&text={suburb}&state=South%20Australia&apiKey={geoapify_key}"
    response = requests.get(url).json()
    pprint(response)

    if 'features' in response and len(response['features']) > 0:
        #Get the Latitude and Longitude
        latitude = response['features'][0]['properties']['lat']
        longitude = response['features'][0]['properties']['lon']
    else:
        latitude, longitude = None, None    
    
    # Append latitude and longitude to the respective lists
    latitude_list.append(latitude)
    longitude_list.append(longitude)
    
# Add latitude and longitude columns to the DataFrame
unique_suburb_postcode_df['Latitude'] = latitude_list
unique_suburb_postcode_df['Longitude'] = longitude_list

# Print the updated DataFrame
unique_suburb_postcode_df.head()

{'features': [{'bbox': [132.0977062, -26.3334536, 132.1890019, -26.2562562],
               'geometry': {'coordinates': [132.1338209, -26.2766919],
                            'type': 'Point'},
               'properties': {'address_line1': 'Pukatja SA 0872',
                              'address_line2': 'Australia',
                              'category': 'administrative',
                              'city': 'Pukatja',
                              'country': 'Australia',
                              'country_code': 'au',
                              'county': 'Anangu Pitjantjatjara Yankunytjatjara',
                              'datasource': {'attribution': '© OpenStreetMap '
                                                            'contributors',
                                             'license': 'Open Database License',
                                             'sourcename': 'openstreetmap',
                                             'url': 'https://www.openstr

,Postcode - Incident,Suburb - Incident,Latitude,Longitude
0,0872,PUKATJA,-26.276692,132.133821
1,2010,DARLINGHURST,-33.878338,151.219225
2,2148,MARAYONG,-33.749167,150.891944
3,2330,SINGLETON,-32.695366,151.022209
4,2469,BONALBO,-28.735609,152.621454


In [72]:
unique_suburb_postcode_df.head()

,Postcode - Incident,Suburb - Incident,Latitude,Longitude
0,0872,PUKATJA,-26.276692,132.133821
1,2010,DARLINGHURST,-33.878338,151.219225
2,2148,MARAYONG,-33.749167,150.891944
3,2330,SINGLETON,-32.695366,151.022209
4,2469,BONALBO,-28.735609,152.621454


In [73]:
# Group the DataFrame by 'Postcode - Incident' and aggregate the 'Suburb - Incident' column
# Group the DataFrame by 'Postcode - Incident' and aggregate the 'Suburb - Incident' and 'Offence count' columns
post_sub = df.groupby('Postcode - Incident').agg({'Suburb - Incident': lambda x: ', '.join(set(x)), 'Offence count': 'sum'}).reset_index()

# Rename the aggregated columns
post_sub.rename(columns={'Suburb - Incident': 'Suburbs', 'Offence count': 'Total Incident Count'}, inplace=True)
# Print the new DataFrame
print(post_sub)

    Postcode - Incident                                            Suburbs  \
0                  0872  PIPALYATJARA, IWANTJA, KANPI, AMATA, KALTJITI,...   
1                  2010                                       DARLINGHURST   
2                  2148                                           MARAYONG   
3                  2330                                          SINGLETON   
4                  2469                                            BONALBO   
..                  ...                                                ...   
350                5960                      TORRENS ISLAND, GARDEN ISLAND   
351                6330                                         TORNDIRRUP   
352                6440                                            JAMESON   
353                 872  PIPALYATJARA, IWANTJA, PETERMANN, AMATA, KANPI...   
354       NOT DISCLOSED                                      NOT DISCLOSED   

     Total Incident Count  
0                     723  
1      

In [78]:
merged_latlon = pd.merge(post_sub, unique_suburb_postcode_df, on='Postcode - Incident' )
# Convert 'Latitude' column to numeric and coerce non-numeric values to NaN
merged_latlon['Latitude'] = pd.to_numeric(merged_latlon['Latitude'], errors='coerce')

# Drop rows where 'Latitude' is NaN
merged_latlon = merged_latlon.dropna(subset=['Latitude'])
merged_latlon.head()

#Drop rows with <5 incidents.
merged_latlon = merged_latlon[merged_latlon['Total Incident Count'] > 5]

merged_latlon.head()


,Postcode - Incident,Suburbs,Total Incident Count,Suburb - Incident,Latitude,Longitude
0,0872,"PIPALYATJARA, IWANTJA, KANPI, AMATA, KALTJITI,...",723,PUKATJA,-26.276692,132.133821
14,5000,"MILE END, ADELAIDE, UNLEY",13740,ADELAIDE,-34.928181,138.599931
15,5006,NORTH ADELAIDE,885,NORTH ADELAIDE,-34.908501,138.595350
16,5007,"BROMPTON, BOWDEN, WEST HINDMARSH, WELLAND, HIN...",1119,BOWDEN,-34.902896,138.579979
17,5008,"RENOWN PARK, RIDLEYTON, CROYDON, WEST CROYDON,...",1499,CROYDON PARK,-34.880250,138.567491


In [105]:
%%capture --no-display

# Configure the map
plot = merged_latlon.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "CartoDark",
    frame_width = 1200,
    frame_height = 1000,
    size = "Total Incident Count",
    scale = 0.3,
    color = "Total Incident Count",
    hover_cols = ['Suburbs', 'Postcode - Incident'],
    color_key='rainbow'
)

# Display the map plot
plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Total Incident Count,Suburbs,Postcode - Incident)

In [116]:
#Work out top 5 total crimes
highest_incidents = merged_latlon.sort_values(by='Total Incident Count', ascending=False).head(5)
high_incident_postcodes = highest_incidents['Postcode - Incident'].tolist()
high_incident_postcodes


['5000', '5108', '5112', '5113', '5700']

In [122]:
#I'm sure there is a better way, but... create seperate DF for each of the top 5 postcodes.
df_5000 = df[df['Postcode - Incident'].isin([high_incident_postcodes[0]])]
df_5108 = df[df['Postcode - Incident'].isin([high_incident_postcodes[1]])]
df_5112 = df[df['Postcode - Incident'].isin([high_incident_postcodes[2]])]
df_5113 = df[df['Postcode - Incident'].isin([high_incident_postcodes[3]])]
df_5700 = df[df['Postcode - Incident'].isin([high_incident_postcodes[4]])]